Common imports and global variables for all tasks

In [86]:
import hashlib, time

#Target hashes to find
targets = []
#Dictionary to store the already computed sha256 hash objects
previous_computed_hashes = {}
#Dictionary to store the found passwords
passwords = {}

Common functions for all tasks

In [87]:
#Check if the hash is in our target list, and if so, add it to the password dictionary with the corresponding plain-text input
def check_hash(targets, new_hash, s):
    hash = new_hash.hexdigest()
    if hash in targets:
        passwords[hash] = s

#Iterate over all possible characters using shortlex ordering
def iterate_all_inputs(targets, valid_chars, cur_len, max_len):
    #If we recurse down to a cur_len of 0, then the prev_result for the parent function call should be empty
    if cur_len == 0:
        return ['']

    result = []
    #Recirsevely call iterate_all_inputs, decrementing cur_len each time until we get to 0, appending the result to a list
    for prev_result in iterate_all_inputs(targets, valid_chars, cur_len - 1, max_len):
        #Nested loop to iterate through every char in our valid_chars list
        for cur_char in valid_chars:
            #If the prev_result already ends with the max len of cur_char, skip it
            if not prev_result.endswith(cur_char * max_len):
                #Build the result string with the previous recursion and cur_char and append it to our result list
                s = prev_result + cur_char
                result.append(s)
                
                #Check the current string s against the target hashes
                new_hash = build_hash(s)

                #Check if the hash is valid
                check_hash(targets, new_hash, s)

                #If all passwords found, break early and do not check for longer passwords
                if len(passwords) == len(targets):
                    return result
    return result

#Find x, which is the input s shortened by 1 character by each iteration so we can query an existing hash object
def find_prev_hash_input(s):
    iteration_threshold = 0
    for i in range(1, len(s) - iteration_threshold):
        x = s[:-i]
        if x in previous_computed_hashes:
            return x

#Function to check the current word in the dictionary against the list of targets and construct the new hash object either from scratch or piggy-backing from previously generated hash objects
def build_hash(s):
    new_hash = None
    
    #Get a previous hash input, if it exists
    x = find_prev_hash_input(s)
    #If no previous hash was found, create one from scratch
    if x == None:
        new_hash = hashlib.sha256(s.encode())
    else:
        #print(x + " - " + s)
        prev = previous_computed_hashes[x]
        #Make a copy as to not mutate existing dictionary entries
        new_hash = prev.copy()

        #Update the value of the hash for s by only selecting the "new" characters that are in addition to the value of x
        new_hash.update(s[len(s) - (len(s) - len(x)):].encode())

    #We should now have a starting hash object, either computed from scratch or starting from a previous chunk of input s
    #Finally insert this new hash into the dictonary for the next iteration
    previous_computed_hashes[s] = new_hash

    return new_hash

Code for Task 1

In [88]:
#Reset globals for next task
targets = [
    '594e519ae499312b29433b7dd8a97ff068defcba9755b6d5d00e84c524d67b06',
    'ade5880f369fd9765fb6cffdf67b5d4dfb2cf650a49c848a0ce7be1c10e80b23',
    '83cf8b609de60036a8277bd0e96135751bbc07eb234256d4b65b893360651bf2',
    '0d335a3bea76dac4e3926d91c52d5bdd716bac2b16db8caf3fb6b7a58cbd92a7'
]
passwords = {}
previous_computed_hashes = {}

def task1(targets, size):
    #Build list of valud input characters for the algorithm to check against
    valid_chars = [
        'a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r',
        's','t','u','v','w','x','y','z','0','1','2','3','4','5','6','7','8','9'
    ]
    
    #Call the recursive iteration function to check every permutation of possible input characters, given targets and a max size
    iterate_all_inputs(targets, valid_chars, size, size)
    
    #If all passwords found, print the success
    if len(passwords) == len(targets):
        print("FOUND ALL PASSWORDS")

    #Return the list of found passwords
    return passwords

task1(targets, 4)
for k, v in passwords.items():
    print(v + " - " + k)

FOUND ALL PASSWORDS
z - 594e519ae499312b29433b7dd8a97ff068defcba9755b6d5d00e84c524d67b06
1m - ade5880f369fd9765fb6cffdf67b5d4dfb2cf650a49c848a0ce7be1c10e80b23
999 - 83cf8b609de60036a8277bd0e96135751bbc07eb234256d4b65b893360651bf2
admn - 0d335a3bea76dac4e3926d91c52d5bdd716bac2b16db8caf3fb6b7a58cbd92a7


Code for Task 2

In [89]:
#Reset globals for next task
targets = [
    '1a7648bc484b3d9ed9e2226d223a6193d64e5e1fcacd97868adec665fe12b924',
    '8c6976e5b5410415bde908bd4dee15dfb167a9c873fc4bb8a81f6f2ab448a918',
    '48054a90032bf1348452fd74f3500ef8d2318d9b5582b07449b3b59db841eecd',
    '09537eae89936399905661760584b19f6ff3af4bb807cee0bb663f64b07eea8e',
    'e7798dc61be73b717402d76cbfaaef41c36c85c027a59abd74abbc8c8288bd4f',
    '0f42bcbeedf89160a6cf7ccafe68080f2aafb73b3ef057df6b5e22f1294d0a10',
    '13989fe9c124d4dfca4e2661dcf8449f49a76fb69f9725612a130622ff3f9bfb',
    'd780c9776eb7d602c805af9ed7aa78225b36af0decb6be51045dcbfa661594a3',
    'd2d03c10a4f2c361dbeff74dab0019264e37336f9ef04831943d0f07c0ad52c7',
    'cbb05a10a2fc5cc96ce5da00a12acc54f594eadb85363de665f3e5dcb81d0e51'
]
passwords = {}
previous_computed_hashes = {}

#Main function to loop through all passwords in the dictionary
def brute_force(targets):
    with open("PasswordDictionary.txt", "r") as f:
        for line in f:
            #Check the current string line against the target hashes
            s = line.rstrip("\n")
            new_hash = build_hash(s)

            #Check if the hash is valid
            check_hash(targets, new_hash, s)

            #If all passwords found, break early and do not check for longer passwords
            if len(passwords) == len(targets):
                print("FOUND ALL PASSWORDS")
                break
    return passwords

brute_force(targets)
for k, v in passwords.items():
    print(v + " - " + k)

FOUND ALL PASSWORDS
66666 - 1a7648bc484b3d9ed9e2226d223a6193d64e5e1fcacd97868adec665fe12b924
admin - 8c6976e5b5410415bde908bd4dee15dfb167a9c873fc4bb8a81f6f2ab448a918
admin002 - 48054a90032bf1348452fd74f3500ef8d2318d9b5582b07449b3b59db841eecd
amm1n1str@t0r2010 - 09537eae89936399905661760584b19f6ff3af4bb807cee0bb663f64b07eea8e
P@Ssw0rd~ - e7798dc61be73b717402d76cbfaaef41c36c85c027a59abd74abbc8c8288bd4f
Password123* - 0f42bcbeedf89160a6cf7ccafe68080f2aafb73b3ef057df6b5e22f1294d0a10
Q1w2e3r4@ - 13989fe9c124d4dfca4e2661dcf8449f49a76fb69f9725612a130622ff3f9bfb
Qwertyuiop - d780c9776eb7d602c805af9ed7aa78225b36af0decb6be51045dcbfa661594a3
R00t - d2d03c10a4f2c361dbeff74dab0019264e37336f9ef04831943d0f07c0ad52c7
Winner! - cbb05a10a2fc5cc96ce5da00a12acc54f594eadb85363de665f3e5dcb81d0e51


Code for Task 3

In [90]:
#Reset globals for next task
targets = [
    ['915edb4d39ab6d260e3fb7269f5d9f8cfba3fdc998415298af3e6eb94a82e43e', '27fb57e9'],
    ['5ddce1dc316e7914ab6af64ef7c00d8b603fac32381db963d9359c3371a84b3a', 'b7875b4b'],
    ['7e3b02bacd934245aa0cb3ea4d2b2f993a8681a650e38a63175374c28c4a7d0d', 'ec13ab35'],
    ['d3136c0cb931acc938de13ed45926eb8764f9ea64af31be479be157480fd3014', '29b49fce'],
    ['3a9053a077383d11f5963ef0c66b38c7eb8331cdb03bbdcc0e5055307f67331b', 'acdabf8a'],
    ['59c05d8d7b6d29279975141f7329cd77a5dc6942b036f9dfd30cbcb52c320cb4', '64afe39d'],
    ['c93802a2273a13c2b8378f98dda9f166783cbfce508aeaf570ad0b19a906b4d2', 'f0919683'],
    ['e6a9713791c2ffeddbf6c6c395add47e1fc02ae1fa47febbbdfb694ed688ba61', '081b2451'],
    ['e6ec51a2ef933920ac1e6d3d8ba6ffac77fe94bfb79518b03cd9b94a14e97d3e', 'defb64a3'],
    ['fbecd00c62b01135f9e588883e80f2710a354c0eb73a33a2c5ab5602cc85f6ad', '017bb5b7']
]
passwords = {}
previous_computed_hashes = {}

#Check if the hash is in our target list, and if so, add it to the password dictionary with the corresponding plain-text input
def check_hash(targets, new_hash, s):
    for t in targets:
        salted_hash = new_hash.copy()
        salted_hash.update(t[1].encode())
        if salted_hash.hexdigest() == t[0]:
            passwords[salted_hash.hexdigest()] = s
            break

brute_force(targets)
for k, v in passwords.items():
    print(v + " - " + k)

FOUND ALL PASSWORDS
Windows8 - 59c05d8d7b6d29279975141f7329cd77a5dc6942b036f9dfd30cbcb52c320cb4
Windowsserver2008 - c93802a2273a13c2b8378f98dda9f166783cbfce508aeaf570ad0b19a906b4d2
Windowsserver2008r2 - e6a9713791c2ffeddbf6c6c395add47e1fc02ae1fa47febbbdfb694ed688ba61
Winner!@#00 - 915edb4d39ab6d260e3fb7269f5d9f8cfba3fdc998415298af3e6eb94a82e43e
Winner!@#11 - 5ddce1dc316e7914ab6af64ef7c00d8b603fac32381db963d9359c3371a84b3a
Winner!@#22 - 7e3b02bacd934245aa0cb3ea4d2b2f993a8681a650e38a63175374c28c4a7d0d
Winter - d3136c0cb931acc938de13ed45926eb8764f9ea64af31be479be157480fd3014
Wizard - 3a9053a077383d11f5963ef0c66b38c7eb8331cdb03bbdcc0e5055307f67331b
Wolverine1234 - e6ec51a2ef933920ac1e6d3d8ba6ffac77fe94bfb79518b03cd9b94a14e97d3e
Wordpass - fbecd00c62b01135f9e588883e80f2710a354c0eb73a33a2c5ab5602cc85f6ad
